In [ ]:
import os
!pip install groq
from groq import Groq

!pip install textstat
import os
from groq import Groq
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import textwrap
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:


# ---- Set your API key securely ----
os.environ["GROQ_API_KEY"] = "gsk_C5DR" #insert API key here
API_KEY = os.getenv("GROQ_API_KEY")

if not API_KEY:
    raise ValueError("GROQ_API_KEY environment variable not found.")

client = Groq(api_key=API_KEY)

# ---- Prompt Template ----
def build_prompt(podcast_text):
    return f"""
Could you create a concise, polished summary of the podcast text (between 80 and 90 words)?
Keep the sentences very short (8 to 12 words long) and impactful.
Avoid connector words like “despite,” “and,” “however,” or “moreover” — the goal is clarity and punchiness.
The tone should be professional and engaging, suitable for a LinkedIn carousel post.
Do not include keywords, hashtags, or final takeaways.
Focus purely on the **main topics** covered in the podcast, and **mention the speaker by name** where relevant.
Avoid summarizing the podcast like a biography.
Make the summary captivating and compelling to attract a large audience.

Podcast Text:
{podcast_text}
"""

# ---- Chunking Large Input ----
def chunk_text(text, max_chars=3000, overlap=200):
    """
    Splits the input text into overlapping chunks to prevent semantic loss.
    """
    words = text.split()
    chunks = []
    start = 0
    while start < len(words):
        end = start + max_chars
        chunk = ' '.join(words[start:end])
        chunks.append(chunk)
        start += max_chars - overlap
    return chunks

# ---- Summarization API Call ----
def summarize_text(text):
    prompt = build_prompt(text)
    response = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        #model="gemma2-9b-it",
        model = "llama-3.3-70b-versatile"
    )
    return response.choices[0].message.content.strip()

# ---- Summarize Full Podcast ----
def summarize_podcast_full(podcast_text):
    chunks = chunk_text(podcast_text)
    partial_summaries = [summarize_text(chunk) for chunk in chunks]

    combined_summary = " ".join(partial_summaries)
    final_summary = summarize_text(combined_summary)

    return final_summary, partial_summaries

# ---- Evaluation Metrics ----
def evaluate_summary(original_text, summary_text):
    vectorizer = TfidfVectorizer().fit_transform([original_text, summary_text])
    similarity = cosine_similarity(vectorizer[0:1], vectorizer[1:2])[0][0]

    compression_ratio = round(len(summary_text) / len(original_text), 3)

    try:
        import textstat
        readability = textstat.flesch_reading_ease(summary_text)
    except ImportError:
        readability = "Requires textstat library (pip install textstat)"

    return {
        "Compression Ratio": compression_ratio,
        "Coverage Score (cosine similarity)": round(similarity, 3),
        "Readability Score": readability
    }

# ---- File I/O ----
def load_podcast_text(file_path):
    with open(file_path, 'r') as file:
        return file.read()

def save_summary_to_file(summary, output_file_path):
    with open(output_file_path, 'w') as file:
        file.write(summary)

# ---- Optional: Driver Function ----
def main(input_path, output_path):
    podcast_text = load_podcast_text(input_path)
    summary, parts = summarize_podcast_full(podcast_text)
    save_summary_to_file(summary, output_path)

    metrics = evaluate_summary(podcast_text, summary)
    print("\nEvaluation Metrics:")
    for k, v in metrics.items():
        print(f"{k}: {v}")

# Example usage:
# main("podcast.txt", "summary.txt")


In [ ]:
#file_path = 'latest_tracscript.txt'  # Input podcast text file path
#file_path = "latest_second_transcript.txt"
file_path = "latest_third_transcript.txt"
#output_file_path = 'podcast_summary_chunking_gemma2-9b-it_thirdnewtrial.txt'
#output_file_path = 'podcast_summary_chunking_gemma2-9b-it_secondnewtrial.txt'
#output_file_path = 'podcast_summary_chunking_llama-3.3-70b-versatile_firstnewtrial.txt'
output_file_path = 'podcast_summary_chunking_llama-3.3-70b-versatile_thirdnewtrial.txt'
main(file_path, output_file_path)

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jtp123avetdv902ctn1sr6r6` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 96417, Requested 4282. Please try again in 10m3.891999999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

NameError: name 'summarize_podcast' is not defined